In [2]:
import cv2
import numpy as np
import dlib
import playsound
from threading import Thread

In [4]:
path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predict_landmark = dlib.shape_predictor(path)

In [5]:
# function transform landmark to array
def landmark_transform(landmarks):
    land_mark_array = []
    for i in landmarks:
        land_mark_array.append([int(i.x),int(i.y)])
    return np.array(land_mark_array)

In [7]:
# tinh toán distance
def calculate_distance(eye):
    assert len(eye)== 6
    p0,p1,p2,p3,p4,p5 = eye
    distance_p1_p5 = np.sqrt((p1[0]-p5[0])**2 + (p1[1]-p5[1])**2)
    distance_p2_p4 = np.sqrt((p2[0]-p4[0])**2 + (p2[1]-p4[1])**2)
    distance_p0_p3 = np.sqrt((p0[0]-p3[0])**2 + (p0[1]-p3[1])**2)
    EAR = (distance_p1_p5+distance_p2_p4)/(2*distance_p0_p3)
    return EAR

In [8]:
# function draw contours of eyes
def draw_contours(image,cnt):
    hull = cv2.convexHull(cnt)
    cv2.drawContours(image,[hull],-1,(0,255,0),2)

In [9]:
def sound_alarm():
    playsound.playsound("sound.mp3")

In [10]:
cap = cv2.VideoCapture(0)
total=0
alarm=False
while cap.isOpened() == True :
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    rects = detector(frame_gray,1)
    if len(rects) > 0 :
        for i in rects:
            cv2.rectangle(frame,(i.left(),i.top()),(i.right(),i.bottom()),(0,255,0),2)
            land_mark = predict_landmark(frame_gray,i)
            left_eye = landmark_transform(land_mark.parts()[36:42])
            right_eye = landmark_transform(land_mark.parts()[42:48])
            draw_contours(frame,left_eye)
            draw_contours(frame,right_eye)
            EAR_left,EAR_right = calculate_distance(left_eye),calculate_distance(right_eye)
            ear = np.round((EAR_left+EAR_right)/2,2)
            #print(ear)
            if ear > 0.25 :
                total=0
                alarm=False
                cv2.putText(frame, "Eyes Open ", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255, 0 ), 2)
            else:
                total+=1
                print(total)
                if total>20:
                    if not alarm:
                        sound_alarm()
                        cv2.putText(frame, "drowsiness detect" ,(10, 30),cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
            cv2.putText(frame, "EAR :"+str(ear) ,(200, 100),cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 255, 0), 2)           
    cv2.imshow("image", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break                        